In [0]:
# SOLUTION BLOCK (for reference - implement the TODOs above first!)
# Uncomment and run this cell if you need help completing the Bronze layer

"""
from pyspark.sql import functions as F

# Many CVE v5 exports have metadata under \"cveMetadata.datePublished\"
# Fall back defensively if a different field name appears
date_col = F.col(\"cveMetadata.datePublished\").cast(\"string\")
df_raw = df_raw.withColumn(\"_datePublished_ts\", F.to_timestamp(date_col))

# 2) Filter to 2024 publications
df_2024 = df_raw.filter(F.year(F.col(\"_datePublished_ts\")) == 2024)

# 3) Data quality gates
cnt_all = df_raw.count()
cnt_2024 = df_2024.count()
null_ids = df_2024.filter(F.col(\"cveMetadata.cveId\").isNull()).count()
distinct_ids = df_2024.select(\"cveMetadata.cveId\").distinct().count()

print(f\"Raw total: {cnt_all:,}\")
print(f\"2024 total: {cnt_2024:,}\")
print(f\"Null cveId (2024): {null_ids}\")
print(f\"Distinct cveId (2024): {distinct_ids:,}\")

assert cnt_2024 >= 30000, \"Too few 2024 rows—did the filter or read fail?\"
assert null_ids == 0, \"cveId should not be null\"
assert distinct_ids == cnt_2024, \"cveId expected to be unique in 2024\"

# 4) Write Delta + register table
(df_2024
 .repartition(8)
 .write
 .format(\"delta\")
 .mode(\"overwrite\")
 .save(bronze_path))

spark.sql(f\"DROP TABLE IF EXISTS {table_name}\")
spark.sql(f\"CREATE TABLE {table_name} USING DELTA LOCATION '{bronze_path}'\")

print(\"✅ Bronze Delta written and table registered:\", table_name)

# 5) Verification for screenshots
spark.read.format(\"delta\").load(bronze_path).printSchema()
spark.sql(f\"DESCRIBE DETAIL {table_name}\").show(truncate=False)
"""

'\nfrom pyspark.sql import functions as F\n\n# Many CVE v5 exports have metadata under "cveMetadata.datePublished"\n# Fall back defensively if a different field name appears\ndate_col = F.col("cveMetadata.datePublished").cast("string")\ndf_raw = df_raw.withColumn("_datePublished_ts", F.to_timestamp(date_col))\n\n# 2) Filter to 2024 publications\ndf_2024 = df_raw.filter(F.year(F.col("_datePublished_ts")) == 2024)\n\n# 3) Data quality gates\ncnt_all = df_raw.count()\ncnt_2024 = df_2024.count()\nnull_ids = df_2024.filter(F.col("cveMetadata.cveId").isNull()).count()\ndistinct_ids = df_2024.select("cveMetadata.cveId").distinct().count()\n\nprint(f"Raw total: {cnt_all:,}")\nprint(f"2024 total: {cnt_2024:,}")\nprint(f"Null cveId (2024): {null_ids}")\nprint(f"Distinct cveId (2024): {distinct_ids:,}")\n\nassert cnt_2024 >= 30000, "Too few 2024 rows—did the filter or read fail?"\nassert null_ids == 0, "cveId should not be null"\nassert distinct_ids == cnt_2024, "cveId expected to be unique in 20

In [0]:
# ==============================================================
# 🪙 CVE LAKEHOUSE – BRONZE LAYER (FULL DATASET, UC-SAFE)
# - Reads all CVE JSON files (~38 K)
# - Filters 2024 records
# - Flattens Structs + serializes Arrays of Structs to JSON
# - Writes Delta files to /Volumes/workspace/default/assignment1/bronze
# - Verifies by direct Delta read (no CREATE TABLE)
# ==============================================================

from pyspark.sql import functions as F, types as T
import re, time

# ---- Configuration ----
volume_root      = "/Volumes/workspace/default/assignment1"
volume_cves_dir  = f"{volume_root}/cve/json_files"
bronze_path      = f"{volume_root}/bronze"      # final Delta output
spark.conf.set("spark.sql.shuffle.partitions", "8")

print("🎯 Starting CVE Bronze Layer (FULL RUN – UC Safe)\n")

# ---------------------------------------------------------------
# 🧩 Helper – recursive sanitizer (unchanged from tested subset)
# ---------------------------------------------------------------
INVALID_CHARS = re.compile(r"[ ,;{}()\n\t=]")

def sanitize_path(path: str) -> str:
    return INVALID_CHARS.sub("_", path)

def build_sanitized_columns(df):
    cols = []
    def walk(schema, prefix=""):
        for field in schema.fields:
            name = field.name
            path = f"{prefix}.{name}" if prefix else name
            safe = sanitize_path(path)
            dt = field.dataType
            if isinstance(dt, T.StructType):
                walk(dt, path)
            elif isinstance(dt, T.ArrayType) and isinstance(dt.elementType, T.StructType):
                cols.append(F.to_json(F.col(path)).alias(safe))
            else:
                cols.append(F.col(path).alias(safe))
    walk(df.schema)
    return cols

# ---------------------------------------------------------------
# 1️⃣ Read ALL JSON files recursively
# ---------------------------------------------------------------
print("📖 Reading ALL JSON files recursively from volume...")
t0 = time.time()
df_raw = (
    spark.read
        .option("multiLine", True)
        .option("recursiveFileLookup", "true")
        .json(volume_cves_dir)
        .withColumn("_source_file", F.col("_metadata.file_path"))
)
raw_count = df_raw.count()
print(f"✅ Raw CVE records loaded: {raw_count:,} files in {time.time()-t0:,.1f}s")

# ---------------------------------------------------------------
# 2️⃣ Filter records published in 2024
# ---------------------------------------------------------------
print("\n📆 Filtering records for 2024 ...")
df_2024 = (
    df_raw
        .withColumn("datePublished_ts", F.to_timestamp(F.col("cveMetadata.datePublished")))
        .filter(F.year(F.col("datePublished_ts")) == 2024)
        .withColumn("_ingest_ts", F.current_timestamp())
        .drop("datePublished_ts")
)
count_2024 = df_2024.count()
print(f"✅ 2024 CVE records count: {count_2024:,}")

# ---------------------------------------------------------------
# 3️⃣ Data Quality Checks
# ---------------------------------------------------------------
print("\n🧪 Running Data Quality Checks ...")
null_ids     = df_2024.filter(F.col("cveMetadata.cveId").isNull()).count()
distinct_ids = df_2024.select("cveMetadata.cveId").distinct().count()

print("🔍 DQ Summary:")
print(f"   • Total 2024 rows : {count_2024:,}")
print(f"   • Null CVE IDs   : {null_ids:,}")
print(f"   • Distinct CVE IDs: {distinct_ids:,}")

assert null_ids == 0, "❌ Found null CVE IDs"
assert distinct_ids == count_2024, "❌ Duplicate CVE IDs found"
print("✅ Basic DQ checks passed.")

# ---------------------------------------------------------------
# 4️⃣ Flatten + sanitize for Delta
# ---------------------------------------------------------------
print("\n🧼 Flattening and sanitizing column names ...")
sanitized_cols = build_sanitized_columns(df_2024)
df_clean = df_2024.select(sanitized_cols)
print(f"✅ Sanitized column count: {len(df_clean.columns)}")

# ---------------------------------------------------------------
# 5️⃣ Write to Delta
# ---------------------------------------------------------------
print("\n💾 Writing full 2024 dataset to Delta ...")
(
    df_clean.write
        .format("delta")
        .mode("overwrite")
        .save(bronze_path)
)
print(f"✅ Delta write complete at: {bronze_path}")

# ---------------------------------------------------------------
# 6️⃣ Verify by reading back (no CREATE TABLE)
# ---------------------------------------------------------------
print("\n🧾 Reading Delta back for verification ...")
df_check = spark.read.format("delta").load(bronze_path)

print(f"✅ Read-back successful. Record count: {df_check.count():,}")
print("\n📐 Schema of final Bronze Delta :")
df_check.printSchema()

print("\n📊 Sample records (for screenshot):")
display(df_check.limit(10))

print("\n📁 Files in Bronze Delta path :")
display(dbutils.fs.ls(bronze_path))

print("\n📸 REQUIRED SCREENSHOTS:")
print("  • df_check.count() ≈ 38 000–40 000 records")
print("  • DQ Summary output above")
print("  • dbutils.fs.ls(bronze_path) showing _delta_log + .parquet files")


🎯 Starting CVE Bronze Layer (FULL RUN – UC Safe)

📖 Reading ALL JSON files recursively from volume...
✅ Raw CVE records loaded: 38,727 files in 2,295.6s

📆 Filtering records for 2024 ...
✅ 2024 CVE records count: 32,924

🧪 Running Data Quality Checks ...
🔍 DQ Summary:
   • Total 2024 rows : 32,924
   • Null CVE IDs   : 0
   • Distinct CVE IDs: 32,924
✅ Basic DQ checks passed.

🧼 Flattening and sanitizing column names ...
✅ Sanitized column count: 104

💾 Writing full 2024 dataset to Delta ...
✅ Delta write complete at: /Volumes/workspace/default/assignment1/bronze

🧾 Reading Delta back for verification ...
✅ Read-back successful. Record count: 32,924

📐 Schema of final Bronze Delta :
root
 |-- containers.adp: string (nullable = true)
 |-- containers.cna.affected: string (nullable = true)
 |-- containers.cna.configurations: string (nullable = true)
 |-- containers.cna.cpeApplicability: string (nullable = true)
 |-- containers.cna.credits: string (nullable = true)
 |-- containers.cna.date

containers.adp,containers.cna.affected,containers.cna.configurations,containers.cna.cpeApplicability,containers.cna.credits,containers.cna.dateAssigned,containers.cna.datePublic,containers.cna.descriptions,containers.cna.exploits,containers.cna.impacts,containers.cna.metrics,containers.cna.problemTypes,containers.cna.providerMetadata.dateUpdated,containers.cna.providerMetadata.orgId,containers.cna.providerMetadata.shortName,containers.cna.references,containers.cna.rejectedReasons,containers.cna.replacedBy,containers.cna.solutions,containers.cna.source.advisory,containers.cna.source.defect,containers.cna.source.defects,containers.cna.source.discovery,containers.cna.source.lang,containers.cna.source.value,containers.cna.tags,containers.cna.timeline,containers.cna.title,containers.cna.workarounds,containers.cna.x_affectedList,containers.cna.x_generator.engine,containers.cna.x_generator.env,containers.cna.x_generator.origin,containers.cna.x_legacyV4Record.CNA_private.Current-Status,containers.cna.x_legacyV4Record.CNA_private.Priority,containers.cna.x_legacyV4Record.CNA_private.STATE,containers.cna.x_legacyV4Record.CNA_private.TYPE,containers.cna.x_legacyV4Record.CNA_private.affectedKeywords,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.affected,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.appliesTo,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.product_versions,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.unaffected,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.unknown,containers.cna.x_legacyV4Record.CNA_private.owner,containers.cna.x_legacyV4Record.CNA_private.publish.month,containers.cna.x_legacyV4Record.CNA_private.publish.year,containers.cna.x_legacyV4Record.CNA_private.publish.ym,containers.cna.x_legacyV4Record.CNA_private.share_with_CVE,containers.cna.x_legacyV4Record.CNA_private.show_cvss,containers.cna.x_legacyV4Record.CVE_data_meta.ASSIGNER,containers.cna.x_legacyV4Record.CVE_data_meta.DATE_PUBLIC,containers.cna.x_legacyV4Record.CVE_data_meta.ID,containers.cna.x_legacyV4Record.CVE_data_meta.STATE,containers.cna.x_legacyV4Record.CVE_data_meta.TITLE,containers.cna.x_legacyV4Record.affects.vendor.vendor_data,containers.cna.x_legacyV4Record.credit,containers.cna.x_legacyV4Record.data_format,containers.cna.x_legacyV4Record.data_type,containers.cna.x_legacyV4Record.data_version,containers.cna.x_legacyV4Record.description.description_data,containers.cna.x_legacyV4Record.exploit,containers.cna.x_legacyV4Record.generator.engine,containers.cna.x_legacyV4Record.impact.cvss.Automatable,containers.cna.x_legacyV4Record.impact.cvss.Recovery,containers.cna.x_legacyV4Record.impact.cvss.Safety,containers.cna.x_legacyV4Record.impact.cvss.attackComplexity,containers.cna.x_legacyV4Record.impact.cvss.attackRequirements,containers.cna.x_legacyV4Record.impact.cvss.attackVector,containers.cna.x_legacyV4Record.impact.cvss.baseScore,containers.cna.x_legacyV4Record.impact.cvss.baseSeverity,containers.cna.x_legacyV4Record.impact.cvss.privilegesRequired,containers.cna.x_legacyV4Record.impact.cvss.providerUrgency,containers.cna.x_legacyV4Record.impact.cvss.subAvailabilityImpact,containers.cna.x_legacyV4Record.impact.cvss.subConfidentialityImpact,containers.cna.x_legacyV4Record.impact.cvss.subIntegrityImpact,containers.cna.x_legacyV4Record.impact.cvss.userInteraction,containers.cna.x_legacyV4Record.impact.cvss.valueDensity,containers.cna.x_legacyV4Record.impact.cvss.vectorString,containers.cna.x_legacyV4Record.impact.cvss.version,containers.cna.x_legacyV4Record.impact.cvss.vulnAvailabilityImpact,containers.cna.x_legacyV4Record.impact.cvss.vulnConfidentialityImpact,containers.cna.x_legacyV4Record.impact.cvss.vulnIntegrityImpact,containers.cna.x_legacyV4Record.impact.cvss.vulnerabilityResponseEffort,containers.cna.x_legacyV4Record.problemtype.problemtype_data,containers.cna.x_legacyV4Record.references.reference_data,containers.cna.x_legacyV4Record.solution,containers.cna.x_legacyV4Record.so


📁 Files in Bronze Delta path :


path,name,size,modificationTime
dbfs:/Volumes/workspace/default/assignment1/bronze/_delta_log/,_delta_log/,0,1762836593612
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00000-e8071914-0002-42b1-999b-f9257168ff28.c000.snappy.parquet,part-00000-e8071914-0002-42b1-999b-f9257168ff28.c000.snappy.parquet,3698405,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00001-d2b889da-10eb-4c33-92e0-9cbace545568.c000.snappy.parquet,part-00001-d2b889da-10eb-4c33-92e0-9cbace545568.c000.snappy.parquet,4957383,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00002-90c6ea31-75df-4216-8adc-01a564b0892d.c000.snappy.parquet,part-00002-90c6ea31-75df-4216-8adc-01a564b0892d.c000.snappy.parquet,4903939,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00003-a50c1550-5552-42b0-9dc5-c4b385a0bb96.c000.snappy.parquet,part-00003-a50c1550-5552-42b0-9dc5-c4b385a0bb96.c000.snappy.parquet,4740399,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00004-5ec1fac5-8431-4267-8f08-7661ba2f9c6f.c000.snappy.parquet,part-00004-5ec1fac5-8431-4267-8f08-7661ba2f9c6f.c000.snappy.parquet,3651867,1762836590000



📸 REQUIRED SCREENSHOTS:
  • df_check.count() ≈ 38 000–40 000 records
  • DQ Summary output above
  • dbutils.fs.ls(bronze_path) showing _delta_log + .parquet files


# Assignment 1: Bronze Layer Example - CVE 2024 Data Ingestion
**DIC 587 - Data Intensive Computing - Fall 2025**

This notebook implements the Bronze layer of our medallion architecture:
- Downloads CVEProject/cvelistV5 repository 
- Filters to 2024 vulnerabilities only (~40,000 records)
- Stores raw JSON as Delta tables with ACID guarantees
- Demonstrates streaming data ingestion patterns

**Learning Objectives:**
- Understand Bronze layer concepts (raw data preservation)
- Practice JSON schema-on-read with PySpark
- Learn Delta Lake table registration
- Handle large-scale data downloads programmatically

## Step 1: Environment Setup and Data Download

In [0]:
import io, os, zipfile, urllib.request, json, shutil
from pyspark.sql.functions import input_file_name, col, to_timestamp, year

# Configuration
USE_OFFLINE = False  # Set True if you manually uploaded /FileStore/cvelistV5.zip
TARGET_YEAR = 2024   # Focus on 2024 CVEs only

# Paths - using /tmp and /FileStore to avoid Community Edition I/O errors
base_dir = "/tmp/cve"
bronze_path = "dbfs:/FileStore/cve/bronze"
os.makedirs(base_dir, exist_ok=True)

print(f"🎯 Target: {TARGET_YEAR} CVE data (~40,000 records)")
print(f"📁 Bronze location: {bronze_path}")

🎯 Target: 2024 CVE data (~40,000 records)
📁 Bronze location: dbfs:/FileStore/cve/bronze


## Step 2: Download CVE Repository

This uploads the entire CVE catalog into a databricks catalog.  You need to figure out how to filter it down to the 2024 records.  You also need to do the other medallion letters!

In [0]:
  import os
  import zipfile
  import urllib.request
  import shutil
  from pyspark.sql.functions import col, to_timestamp, year

  # Configuration
  USE_OFFLINE = False
  TARGET_YEAR = 2024
  volume_root = "/Volumes/workspace/default/assignment1"
  volume_cves_dir = f"{volume_root}/cve/json_files"

  # Check if data already exists in volume FIRST (before downloading!)
  skip_all = False
  try:
      existing_files = dbutils.fs.ls(volume_cves_dir)
      file_count = len([f for f in existing_files if f.name.endswith('.json')])
      
      if file_count > 0:
          print("="*60)
          print(f"✅ Found {file_count} existing JSON files in volume")
          print(f"   📁 Location: {volume_cves_dir}")
          print(f"   ⏩ SKIPPING download, extraction, and copy")
          print(f"   💡 To force re-download, delete the volume directory first:")
          print(f"      dbutils.fs.rm('{volume_cves_dir}', recurse=True)")
          print("="*60)
          print()
          skip_all = True
  except:
      print("📋 No existing data found in volume - proceeding with download...")
      pass

  # Only download and extract if we don't have data
  if not skip_all:
      # Your existing download/extract pattern
      zip_dest = "/tmp/cve/cvelistV5.zip"
      src_extract_dir = "/tmp/cve/cvelistV5-main"

      # Create local temp directory
      os.makedirs("/tmp/cve", exist_ok=True)

      if not USE_OFFLINE:
          zip_url = "https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip"
          print(f"🌐 Downloading: {zip_url}")
          print("📈 This is live streaming data - updates every ~7 minutes!")

          with urllib.request.urlopen(zip_url) as r:
              data = r.read()
          with open(zip_dest, "wb") as f:
              f.write(data)

          print(f"✅ Downloaded {len(data):,} bytes")
      else:
          zip_dest = "/dbfs/FileStore/cvelistV5.zip"  # uploaded by you
          assert os.path.exists(zip_dest), "❌ Upload /FileStore/cvelistV5.zip or set USE_OFFLINE=False"
          print("📁 Using offline ZIP file")

      # Clean previous extracts
      if os.path.exists(src_extract_dir):
          shutil.rmtree(src_extract_dir, ignore_errors=True)

      # Extract ZIP
      print("📦 Extracting ZIP archive...")
      with zipfile.ZipFile(zip_dest) as z:
          z.extractall("/tmp/cve/")

      src_glob = "/tmp/cve/cvelistV5-main/cves"
      print(f"📂 Found local JSON directory: {src_glob}")

      src_glob = "/tmp/cve/cvelistV5-main/cves"
      print(f"📂 Found local JSON directory: {src_glob}")

      # Copy JSON files to Unity Catalog volume for Spark access
      print("📋 Copying JSON files to Unity Catalog volume...")
      print("   (Required because Spark cannot access local filesystem)")

      # Remove existing volume directory
      try:
          dbutils.fs.rm(volume_cves_dir, recurse=True)
      except:
          pass

      dbutils.fs.mkdirs(volume_cves_dir)

      # Copy ONLY 2024 JSON files from local to volume (optimized!)
      json_count = 0
      
      # Target only the 2024 directory for efficiency
      target_2024_dir = os.path.join(src_glob, "2024")      
      if os.path.exists(target_2024_dir):
          print(f"   📂 Found 2024 directory: {target_2024_dir}")
          print(f"   🎯 Only copying 2024 CVEs for efficiency...")
          
          for root, dirs, files in os.walk(target_2024_dir):
              for file in files:
                  if file.endswith('.json'):
                      local_file_path = os.path.join(root, file)

                      # Read file content
                      with open(local_file_path, 'r', encoding='utf-8') as f:
                          content = f.read()

                      # Write to volume with unique name (include subdirectory info)
                      relative_path = os.path.relpath(local_file_path, src_glob)
                      safe_filename = relative_path.replace('/', '_').replace('\\', '_')
                      volume_file_path = f"{volume_cves_dir}/{safe_filename}"

                      dbutils.fs.put(volume_file_path, content, overwrite=True)
                      json_count += 1

                      if json_count % 1000 == 0:
                          print(f"      Copied {json_count} JSON files...")
      else:
          print(f"   ⚠️  2024 directory not found, copying all CVEs (slower)...")
          # Fallback: copy all files if 2024 directory doesn't exist
          for root, dirs, files in os.walk(src_glob):
              for file in files:
                  if file.endswith('.json'):
                      local_file_path = os.path.join(root, file)

                      # Read file content
                      with open(local_file_path, 'r', encoding='utf-8') as f:
                          content = f.read()

                      # Write to volume with unique name (include subdirectory info)
                      relative_path = os.path.relpath(local_file_path, src_glob)
                      safe_filename = relative_path.replace('/', '_').replace('\\', '_')
                      volume_file_path = f"{volume_cves_dir}/{safe_filename}"

                      dbutils.fs.put(volume_file_path, content, overwrite=True)
                      json_count += 1

                      if json_count % 1000 == 0:
                          print(f"      Copied {json_count} JSON files...")

      print(f"✅ Copied {json_count} JSON files to volume (2024 CVEs only)")

  # Read the data (whether existing or just copied)
  print("🔄 Reading JSON files with Spark...")

  # Now read from volume location (accessible to Spark)
  df_raw = (spark.read
            .option("multiLine", True)
            .json(volume_cves_dir))

  # Add source file tracking (Unity Catalog compatible)
  df_raw = df_raw.withColumn("_source_file", col("_metadata.file_path"))

  print(f"📊 Total CVE records found: {df_raw.count():,}")


📋 No existing data found in volume - proceeding with download...
🌐 Downloading: https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip
📈 This is live streaming data - updates every ~7 minutes!
✅ Downloaded 523,000,347 bytes
📦 Extracting ZIP archive...
📂 Found local JSON directory: /tmp/cve/cvelistV5-main/cves
📂 Found local JSON directory: /tmp/cve/cvelistV5-main/cves
📋 Copying JSON files to Unity Catalog volume...
   (Required because Spark cannot access local filesystem)
   📂 Found 2024 directory: /tmp/cve/cvelistV5-main/cves/2024
   🎯 Only copying 2024 CVEs for efficiency...
Wrote 9943 bytes.
Wrote 5334 bytes.
Wrote 15645 bytes.
Wrote 10115 bytes.
Wrote 5284 bytes.
Wrote 7956 bytes.
Wrote 10339 bytes.
Wrote 5354 bytes.
Wrote 8050 bytes.
Wrote 5162 bytes.
Wrote 4928 bytes.
Wrote 4846 bytes.
Wrote 5436 bytes.
Wrote 6664 bytes.
Wrote 4815 bytes.
Wrote 8366 bytes.
Wrote 5073 bytes.
Wrote 4966 bytes.
Wrote 5139 bytes.
Wrote 10116 bytes.
Wrote 5768 bytes.
Wrote 5398 bytes.
Wro

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
len(dbutils.fs.ls("/Volumes/workspace/default/assignment1/cve/json_files/"))

38727